---
Author: Mustapha Bouhsen <br>
[LinkedIn](https://www.linkedin.com/in/mustapha-bouhsen/)<br>
[Git](https://github.com/mus514)<br>
Date: February 14, 2024<br>
---

In [0]:
%run Repos/bouhsen.m@gmail.com/ML_Pipeline_Hub/library/garch_model

In [0]:
%run Repos/bouhsen.m@gmail.com/ML_Pipeline_Hub/library/daily_utilities

## Creating table containg the GARCH(1,1) volatility for each stock

The volatility is given by :

$$
\sigma_{t}^2= \omega+ \alpha y^2_{t-1} + \beta \sigma^2_{t-1} \ , \ \omega \ge 0 \ \ \ \  \alpha, \beta>0 \ and
 \ \alpha+ \beta<1
$$

In [0]:
#-----------------------------------------
# Set the prod folder path
#-----------------------------------------
raw_folder_path = "/mnt/raw/"
prod_folder_path = "/mnt/prod/"

stocks = ["aapl", "amzn", "googl", "msft"]

In [0]:
#-----------------------------------------
# Load the data
#-----------------------------------------
returns = spark.sql("SELECT * FROM stocks_returns").toPandas()

In [0]:
#-----------------------------------------
# Calculate the GARCH(1,1) volatility for each stock
#-----------------------------------------
garch_vol = {}
garch_vol["date"] = returns["date"]
for stock in stocks:
    garch_vol[stock] = forecast_vol(returns[stock])[0][:-1]

garch_vol = pd.DataFrame(garch_vol)

In [0]:
#--------------------------------------------------------
# Convert the data to spark to save it esaly to sql table
#--------------------------------------------------------
# Define the schema
schema = StructType([
    StructField("date", StringType(), True),
    StructField("aapl", FloatType(), True),
    StructField("amzn", FloatType(), True),
    StructField("msft", FloatType(), True),
    StructField("googl", FloatType(), True)
])

garch_vol = spark.createDataFrame(garch_vol, schema)

# Check if the table exists
if spark.catalog.tableExists("stocks_volatility"):
    # Drop the existing table
    spark.sql(f"DROP TABLE stocks_volatility")
    print(f'Dropped table: stocks_volatility')

# Create the table
garch_vol.write.format("parquet").saveAsTable("stocks_volatility")

Dropped table: stocks_volatility


In [0]:
#-----------------------------------------
# Write the garch volatility in the prod
#-----------------------------------------
# Temp folder to save temp parquet files
temp_folder = prod_folder_path + f"temp/"

# write data frame to csv
garch_vol.coalesce(1).write.mode("overwrite").option("header", "True").csv(temp_folder)

#get all files path ending with .parquet
files_paths = get_files_paths_from_folders(temp_folder, ".csv")
            
# Copy parquet files to final destination
ingest_and_transform_to_parquet(files_paths, prod_folder_path, "volatilities")

# delete the temp folder
delete_contents_recursively(temp_folder)

In [0]:
%sql
-- Disply the stocks_returns
SELECT *
FROM stocks_volatility
ORDER BY date DESC
LIMIT 10

date,aapl,amzn,msft,googl
2024-02-12,1.8448957E-4,6.4494414E-4,4.1550378E-4,2.0927316E-4
2024-02-09,1.9253211E-4,6.314657E-4,4.1388778E-4,2.0363413E-4
2024-02-08,2.0006507E-4,7.080605E-4,4.4699386E-4,2.2061924E-4
2024-02-07,2.1096437E-4,7.930064E-4,4.7631405E-4,1.9706559E-4
2024-02-06,2.1713106E-4,8.970418E-4,5.1616976E-4,2.131243E-4
2024-02-05,2.221631E-4,0.0010173343,5.544657E-4,2.131472E-4
2024-02-02,2.3297689E-4,4.126897E-4,5.976942E-4,1.9868766E-4
2024-02-01,2.3329063E-4,3.5980533E-4,6.471031E-4,1.9144946E-4
2024-01-31,2.1728098E-4,3.0829286E-4,2.1628957E-4,1.3339386E-4
2024-01-30,2.000188E-4,2.9780518E-4,2.1344332E-4,1.3986659E-4
